In [1]:
import pandas as pd


ba = pd.read_csv('pobocky/bratislava.csv')
ke = pd.read_csv('pobocky/kosice.csv')
ni = pd.read_csv('pobocky/nitra.csv')
br = pd.read_csv('pobocky/brno.csv')
os = pd.read_csv('pobocky/ostrava.csv')

In [2]:
# 1. Pridanie krajina & mesto
ba['krajina'] = len(ba) * ['Slovensko']
ba['mesto'] = len(ba) * ['Bratislava']

ke['krajina'] = len(ke) * ['Slovensko']
ke['mesto'] = len(ke) * ['Kosice']

ni['krajina'] = len(ni) * ['Slovensko']
ni['mesto'] = len(ni) * ['Nitra']

br['krajina'] = len(br) * ['Cesko']
br['mesto'] = len(br) * ['Brno']

os['krajina'] = len(os) * ['Cesko']
os['mesto'] = len(os) * ['Ostrava']

In [3]:
# 2. Spojenie tabuliek
predaje = pd.concat([ba, ke, ni, br, os])
predaje = predaje.drop('Unnamed: 0', axis=1).reset_index(drop=True)

In [4]:
# 3. Zjednotenie meny na euro
kurz = 24.69

predaje.loc[predaje['krajina'] == 'Cesko', 'cena'] = round((predaje.loc[predaje['krajina'] == 'Cesko', 'cena'] / 24.69), 2)

In [5]:
print(sorted(list(set(predaje['produkt']))))

['Chleb', 'Chlieb', 'Laptop', 'Mleko', 'Mlieko', 'Mobil', 'Sveter', 'Svetr', 'Tablet', 'Tricko', 'Triko']


In [6]:
# 4. Zjednotenie mena produktov
predaje.loc[predaje['produkt'] == 'Chleb', 'produkt'] = 'Chlieb'
predaje.loc[predaje['produkt'] == 'Mleko', 'produkt'] = 'Mlieko'
predaje.loc[predaje['produkt'] == 'Svetr', 'produkt'] = 'Sveter'
predaje.loc[predaje['produkt'] == 'Triko', 'produkt'] = 'Tricko'

In [7]:
# 5. Time format na mesiac => kvartal
predaje['cas_nakupu'] = pd.to_datetime(predaje['cas_nakupu'])
predaje['mesiac'] = predaje['cas_nakupu'].dt.month_name()
predaje['kvartal'] = 'Q' + predaje['cas_nakupu'].dt.quarter.astype(str)

In [8]:
# 6. Delete cas_nakupu
predaje = predaje.sort_values(by='cas_nakupu')
predaje = predaje.drop('cas_nakupu', axis=1)

In [9]:
# 7. Odvodit atribut kategoria (elektronika, potraviny, oblecenie)
import numpy as np


conditions = [
    (predaje['produkt'] == 'Laptop') | (predaje['produkt'] == 'Mobil') | (predaje['produkt'] == 'Tablet'),
    (predaje['produkt'] == 'Chlieb') | (predaje['produkt'] == 'Mlieko'),
    (predaje['produkt'] == 'Sveter') | (predaje['produkt'] == 'Tricko')
]

categories = ['elektronika', 'potraviny', 'oblecenie']

predaje['kategoria'] = np.select(conditions, categories, default='NONE')

In [10]:
# 8. Zmazat atributy navyse
predaje[::int(len(predaje)/12)]

,produkt,cena,krajina,mesto,mesiac,kvartal,kategoria
0,Mlieko,3.00,Slovensko,Bratislava,January,Q1,potraviny
19256,Chlieb,2.00,Slovensko,Kosice,January,Q1,potraviny
2924,Mlieko,3.00,Slovensko,Bratislava,March,Q1,potraviny
47207,Chlieb,1.62,Cesko,Brno,April,Q2,potraviny
48648,Sveter,25.11,Cesko,Brno,May,Q2,oblecenie
66249,Chlieb,1.62,Cesko,Ostrava,June,Q2,potraviny
67537,Chlieb,1.62,Cesko,Ostrava,July,Q3,potraviny
68757,Tricko,10.13,Cesko,Ostrava,August,Q3,oblecenie
54272,Chlieb,1.62,Cesko,Brno,September,Q3,potraviny
55722,Tricko,10.13,Cesko,Brno,October,Q4,oblecenie
